In [ ]:
import os
os.system("./../source/generate_vector_python ~/Projeto/TCC/data-fingerprint/2002_DB1a_data/finger_index.txt  ../data-fingerprint/2002_DB1a_data/Vector_Index/ {} {} {}".format(16,5,0))
os.system("./../source/generate_vector_python ~/Projeto/TCC/data-fingerprint/2002_DB1a_data/finger_search.txt  ../data-fingerprint/2002_DB1a_data/Vector_Search/ {} {} {}".format(16,5,0))

### Criação do índice ElasticSearch

In [153]:
from elasticsearch import Elasticsearch,helpers
def create_index():
    es = Elasticsearch()
    request_body = {
        "settings" : {
            "number_of_shards": 5,
            "number_of_replicas": 1
        },
        "mappings": {
            "hash": {
                "properties": {
                    "value": {"type":"text"},
                    "index_in_hash":{
                        "type": "nested",
                        "properties":{
                            "text":{"type": "text"},
                            "id": {"type": "text"}
                        }
                    }
                    
                }
            }
        }
    }


    es.indices.create(index = 'hashs', body = request_body,request_timeout=10000)
    return 1


create_index()


1

### Inicializando os elementos no índice

In [154]:

%%time
from elasticsearch import Elasticsearch,helpers
es = Elasticsearch()

def init_hashs(num_hash,num_bits):
    
    text = list(map(lambda x: {"text":"", "id":str(x)},list(range(2**num_bits))))

    for pos in range(num_hash):

        yield { "_op_type":"create",
                "_index": "hashs",
                "_type": "hash",
                "_id": str(pos),
                "_source": {
                    "value":str(pos),
                    "index_in_hash":text

               }

        } 
#list(gendata(list_pos_min_1,1))
helpers.bulk(es, init_hashs(192,8),request_timeout=1000)
#a = list(init_hashs(192,8))

CPU times: user 40.2 ms, sys: 203 µs, total: 40.4 ms
Wall time: 301 ms


(192, [])

### Função para gerar text para bulk para indexar ElasticSearch

In [149]:
from elasticsearch import Elasticsearch,helpers
es = Elasticsearch()

def att_hashs(list_positions,id_,id_min):   
    for i,pos in enumerate(list_positions):
        if(pos != 0):
            yield { "_op_type":"update",
                    "_index": "hashs",
                    "_type": "hash",
                    "_id": str(i),
                    "_source": {
                        "script": {
                            "source": " def targets = ctx._source.index_in_hash.findAll(hash_ele -> hash_ele.id == params.hash_index); for(hash_ele in targets){hash_ele.text = hash_ele.text + params.new_text}",
                            "params":{
                                "hash_index":str(pos),
                                "new_text":"P{}_M{} ".format(id_,id_min)
                            }
                        }
                   }
            } 

In [150]:
%%time
import numpy as np
from textwrap import wrap
import os
for x,i,files in os.walk("../data-fingerprint/2002_DB1a_data/Vector_Index/"):
    for ind,file in enumerate(files):
        if(ind ==10):
            break
        print(ind+1,file)
        vector_data = sc.textFile(x + file)
        list_positions = vector_data.map(lambda x:x.replace(" ","")).map(lambda x:wrap(x,8)).map(lambda x: list(map(lambda x1: int(x1,2),x)) ).collect()
        id = int(file.split('_')[0])
        len_list = len(list_positions)
        es = Elasticsearch()
        #list(map( lambda x : helpers.bulk(es,x,request_timeout=1000) , list(map( att_hashs, list_positions, np.repeat(id,len_list), list(range(len_list)) ) ) ) )
        list_for_bulk = list(map( lambda x : list(x) , list(map( att_hashs, list_positions, np.repeat(id,len_list), list(range(len_list)) ) ) ) )
        list_for_bulk = [item for sublist in list_for_bulk for item in sublist]
        helpers.bulk(es,list_for_bulk,request_timeout=1000)

1 26_4.tif.bmp.wsq.txt
2 63_6.tif.bmp.wsq.txt
3 64_1.tif.bmp.wsq.txt
4 12_6.tif.bmp.wsq.txt
5 4_6.tif.bmp.wsq.txt
6 92_6.tif.bmp.wsq.txt
7 53_3.tif.bmp.wsq.txt
8 40_6.tif.bmp.wsq.txt
9 9_2.tif.bmp.wsq.txt
10 49_4.tif.bmp.wsq.txt
CPU times: user 3.32 s, sys: 49.8 ms, total: 3.37 s
Wall time: 51.3 s


### Indexação com função para agrupar conjunto de texto em memória

In [155]:
from elasticsearch import Elasticsearch,helpers
es = Elasticsearch()

def att_hashs(hash_id,i):
    
    for pos,text in enumerate(hash_id):
        if(text != ''):
            yield { "_op_type":"update",
                    "_index": "hashs",
                    "_type": "hash",
                    "_id": str(i),
                    "_source": {
                        "script": {
                            "source": " def targets = ctx._source.index_in_hash.findAll(hash_ele -> hash_ele.id == params.hash_index); for(hash_ele in targets){hash_ele.text = hash_ele.text + params.new_text}",
                            "params":{
                                "hash_index":str(pos),
                                "new_text":text
                            }
                        }
                   }
            } 

def convert_hash_to_bulk(hash):
    return [item for sublist in list(map(lambda x,i:att_hashs(x,i),hash.text_in_hashs,list(range(len(hash.text_in_hashs))))) for item in sublist]

In [156]:
def set_value_in_hash(string_text,index,position):
    global hashs
    
    hashs.text_in_hashs[index][position]+=string_text
    
def generate_word(list_index,id,id_min):
    #id = 1
    return list(map(lambda pos_hash,index_hash: set_value_in_hash("P{}_M{} ".format(id,id_min),index_hash,pos_hash) if(pos_hash > 0) else None ,list_index,list(range(len(list_index)))))

In [163]:
%%time
import numpy as np
from textwrap import wrap
import os
es = Elasticsearch()

for x,i,files in os.walk("../data-fingerprint/2002_DB1a_data/Vector_Index/"):
    hashs = Hash(192,8)
    for ind,file in enumerate(files):
        if(ind %10 == 0 and ind != 0):
            
            list_for_bulk = convert_hash_to_bulk(hashs)
            helpers.bulk(es,list_for_bulk,request_timeout=1000)
            break
            hashs = Hash(192,8)

            
        print(ind+1,file)
        vector_data = sc.textFile(x + file)
        list_positions = vector_data.map(lambda x:x.replace(" ","")).map(lambda x:wrap(x,8)).map(lambda x: list(map(lambda x1: int(x1,2),x)) ).collect()
        id = int(file.split('_')[0])
        len_list = len(list_positions)
        list(map(generate_word, list_positions, np.repeat(id,len_list), list(range(len_list))))

1 26_4.tif.bmp.wsq.txt
2 63_6.tif.bmp.wsq.txt
3 64_1.tif.bmp.wsq.txt
4 12_6.tif.bmp.wsq.txt
5 4_6.tif.bmp.wsq.txt
6 92_6.tif.bmp.wsq.txt
7 53_3.tif.bmp.wsq.txt
8 40_6.tif.bmp.wsq.txt
9 9_2.tif.bmp.wsq.txt
10 49_4.tif.bmp.wsq.txt
CPU times: user 675 ms, sys: 20.2 ms, total: 695 ms
Wall time: 11.3 s


In [16]:
class Hash:
    def __init__(self, num_hashs, num_bits):
        self.text_in_hashs = []
        self.num_bits = num_bits
        self.num_hashs = num_hashs
        self.size_bucket = 2**num_bits
        self.text_in_hashs = list(map(lambda x:list(map(lambda x1: "",list(range(self.size_bucket)))),list(range(self.num_hashs))))

### Final do metódo de indexação com conjunto de texto em memória

In [2]:
import numpy as np
from textwrap import wrap
import os

In [ ]:

def gendata1(list_pos_in_hash):
    list_not_index = [  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  43,  50, 173, 180, 32,  33,  34,  39,  40,  41,  42,  51,  52,  61, 162, 171, 172,
       181, 182, 183, 184, 189, 190, 191]
  #  list_not_index= []
    for i,pos in enumerate(list_pos_in_hash):
        if(pos != 0 and i not in list_not_index):
            yield {"index": "hashs" }
            yield {
                    "_source":"false",

                    "query": {
                        "bool":{
                            "filter":{
                                "bool":{
                                "must":[
                                
                                {"match":{"_id":str(i)}},
                                    
                                    
                                {"nested":{
                                    "path": "index_in_hash",
                                    "query":{
                                        "match": {"index_in_hash.id":str(pos)}},
                                    "inner_hits":{}
                                }}
                                ]
                                }
                            }
                        }
                    }
            }
                            
                            
                            
                            
                            
                            
      

In [54]:
def realize_search(data_consult):
    from elasticsearch import Elasticsearch
    es = Elasticsearch()
    if(data_consult != []):
        data_response = es.msearch(body=data_consult)
        return list(map(lambda x : x['hits']['hits'][0]['inner_hits']['index_in_hash']['hits']['hits'][0]['_source']['text'], data_response['responses']))
    else:
        return []
    

In [55]:

import numpy as np

def func_count_byKey_return_max(x):
    if(x != []):
        import numpy as np
        x_,y_ = np.unique(x,return_counts=True)
        y_max = np.max(y_)
        return " ".join([ i[0].split('_')[0] for i in list(filter(lambda x: x[1] == y_max,list(map(lambda x,y:(x,y) ,x_,y_)))) ])
    else:
        return ""
def func_rank_elements(persons):
    persons_list, count_ = np.unique(persons.split(" "),return_counts=True)
    return persons_list[np.argsort(-1*count_)]


In [87]:
%%time
vector_data = sc.textFile('../data-fingerprint/2002_DB1a_data/Vector_Search/49_1.tif.bmp.wsq.txt')
persons = vector_data.map(lambda x:x.replace(" ","")).map(lambda x:wrap(x,8)).map(lambda x: list(map(lambda x1: int(x1,2),x))).map(lambda x : list(gendata1(x))).map(lambda x: realize_search(x)).map(lambda x : " ".join(x)).map(lambda x : x.split()).map(func_count_byKey_return_max ).reduce(lambda x,y: y + " "+ x).replace("  ", " ")
ranking = func_rank_elements(persons)
pos_  = -1
id = "49"

for p in range(len(ranking)):
    if(id  == ranking[p].split('P')[1]):
        pos_ = p + 1
        break
print(pos_)
#pos_rank[i].append([pos_,file])

25
CPU times: user 14.5 ms, sys: 131 µs, total: 14.6 ms
Wall time: 387 ms


In [89]:
pos_rank =[]
for x,i,files in os.walk("../data-fingerprint/2002_DB1a_data/Vector_Search/"):
    count = 0
    for file in files:
        
       # print(file)
        vector_data = sc.textFile(x + file)
        persons = vector_data.map(lambda x:x.replace(" ","")).map(lambda x:wrap(x,8)).map(lambda x: list(map(lambda x1: int(x1,2),x))).map(lambda x : list(gendata1(x))).map(lambda x: realize_search(x)).map(lambda x : " ".join(x)).map(lambda x : x.split()).map(func_count_byKey_return_max ).reduce(lambda x,y: y + " "+ x).replace("  ", " ")
        ranking = func_rank_elements(persons)
        id = file.split("_")[0]
        pos_  = -1
        for p in range(len(ranking)):
            if(id  == ranking[p].split('P')[1]):
                pos_ = p + 1
                break
        count+=1
        print("count:",count,"filename:",file,"pos_rank:",pos_)
        pos_rank.append([pos_,file])
        

count: 1 filename: 1_6.tif.bmp.wsq.txt pos_rank: 1
count: 2 filename: 91_6.tif.bmp.wsq.txt pos_rank: 1
count: 3 filename: 87_4.tif.bmp.wsq.txt pos_rank: 1
count: 4 filename: 11_7.tif.bmp.wsq.txt pos_rank: 1
count: 5 filename: 58_6.tif.bmp.wsq.txt pos_rank: 1
count: 6 filename: 38_3.tif.bmp.wsq.txt pos_rank: 1
count: 7 filename: 15_8.tif.bmp.wsq.txt pos_rank: 1
count: 8 filename: 10_2.tif.bmp.wsq.txt pos_rank: 2
count: 9 filename: 98_2.tif.bmp.wsq.txt pos_rank: 1
count: 10 filename: 74_4.tif.bmp.wsq.txt pos_rank: 1
count: 11 filename: 87_1.tif.bmp.wsq.txt pos_rank: 1
count: 12 filename: 88_6.tif.bmp.wsq.txt pos_rank: 1
count: 13 filename: 60_1.tif.bmp.wsq.txt pos_rank: 1
count: 14 filename: 4_8.tif.bmp.wsq.txt pos_rank: 1
count: 15 filename: 73_6.tif.bmp.wsq.txt pos_rank: 1
count: 16 filename: 67_1.tif.bmp.wsq.txt pos_rank: 1
count: 17 filename: 57_3.tif.bmp.wsq.txt pos_rank: 1
count: 18 filename: 54_2.tif.bmp.wsq.txt pos_rank: 1
count: 19 filename: 81_5.tif.bmp.wsq.txt pos_rank: 1
coun

KeyboardInterrupt: 

In [121]:
import pandas as pd

df = pd.DataFrame(pos_rank,columns=['pos_rank','file'])

In [123]:
df.shape

(700, 2)

In [124]:
df[(df['pos_rank'] >0) & (df['pos_rank'] <10)].shape

(680, 2)

In [125]:
df.shape

(700, 2)

### Analisando  o índice 

In [21]:
from elasticsearch import Elasticsearch
import  pandas as pd

/home/johnny/.local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [22]:
es = Elasticsearch()
count_index_voids = []
for i in range(192):
    data_r = es.get(index="hashs",id=i,doc_type="hash")
    count_void_elements = 0
    for partial_data in data_r['_source']['index_in_hash']:
        if(partial_data['text'] == ''):
            count_void_elements +=1
 #   if(count_void_elements len(data_r['_source']['index_in_hash']))
    count_index_voids.append([count_void_elements,i])

In [23]:
df = pd.DataFrame(count_index_voids,columns=['voids','index'])


### Existem 20 hashs em que todos os elementos caíram em 0 e foram eliminados

In [24]:
df[df['voids'] >255].shape

(20, 2)

In [25]:
df[df['voids'] >255]['index'].values

array([ 32,  33,  34,  39,  40,  41,  42,  51,  52,  61, 162, 171, 172,
       181, 182, 183, 184, 189, 190, 191])

### Existem 36 hashs que todos os elementos estão na mesma posição, ou seja, são dados agrupados na mesma posição (muito volume) e são irrelevantes

In [5]:
df[df['voids'] ==255].shape

(36, 2)

In [7]:
df[df['voids'] ==255]['index'].values

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  43,  50, 173, 180])